* Gérer les adverbes
* comment mixer les features tf-idf et les autres, pour leur donner du poids? Lire des articles,
* faire deux modèles et mixer
* comment gérer l'ironie?
* Faire une fonction qui va uniquement prédire sur X derniers tweets de la collection tweet

In [66]:
import pymongo as pym
import nltk.data
import re
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils','pour','une','que','quel'] +
            list('@ن%£€‘:&;') + list('abcdefghijklmnopqrstuvwxyz')+
            stop_words.get_stop_words(language='fr')+stopwords.words('french'))


In [77]:
def tweetPreprocessing(collection, nLastTweets=0, retweet=False):

    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(nLastTweets)  
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []

    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue
        # comptes
        a = t['text'].count('!')
        b = t['text'].count('?')
        c = t['text'].count('#')
        d = t['text'].count('"')
        e = t['text'].count('http')
        f = t['text'].count(':')
        g = t['text'].count('»')
        h = t['text'].count('@')
        
        # mot tronqué
        t['text'] = re.sub(r'\w*…', '', t['text'])
        
        # caracteres speciaux
        t['text'] = re.sub(r'\xad', '-',
                           re.sub(r'\n', ' ', # retrait des sauts de ligne
                                  re.sub(r'\[a-zA-Z]*(?!\S)', '', # retrait de ce qui n'est pas un mot
                                         re.sub(r'(?:htt)\S*', '', # retrait des liens http
                                                re.sub(r'^rt.*: ', '', # retrait de la mention retweet
                                                      re.sub(r'\d', '', # retrait des chiffres
                                                              re.sub(r',;!?\/\*(){}«»', ' ', t['text'])))))))
        
        t['text'] = re.sub('|'.join(['’', '_', '/', '-', '\'', '“', '\.']), ' ', t['text'])

        # apostrophes
        t['text'] = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', t['text'])
        negation = 0
        adverbes = 0
        for token in t['text'] : 
            if token in ['ne','pas','n\'','n','sans','aucun','nul','nulle','rien','jamais','personne'] : 
                negation += 1
            if re.match('\w\S*ment', token) is not None :
                adverbes += 1        
        tokenizer = TreebankWordTokenizer()
        t['text'] = tokenizer.tokenize(t['text'])
        t['text'] = [token for token in t['text'] if (token not in stops) and (len(token)>2)]

                
        while '' in t['text']:
            t['text'].pop('')
            
        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
                
            rec = pd.DataFrame([[a, b, c, d, e, f,g, h, negation, adverbes]], 
                               columns=['!', '?', '#', '"', '_http_',':','»','@','négation','adverbes'])
            df = df.append(rec, ignore_index=True)
        
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

def build_feat_mat(df_tweets):
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                use_idf=True, norm='l2', binary=False, min_df=0.001, max_df=0.05, ngram_range=(1,10))
    tfidf = vectorizer.fit_transform(df_tweets['text'].apply(' '.join))
    X = pd.DataFrame(tfidf.toarray())
    X[['!', '?', '#', '"', '_http_',':','»','@','négation','adverbes']] = \
    df_tweets[['!', '?', '#', '"', '_http_',':','»','@','négation','adverbes']]
    return X


def getSentiments(client, retweet, classifier, full_retrain=True): 
    if full_retrain :
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 0, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 0, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
        n_samples, vocabulaire = X.shape
        print("Répartition dans le dataset de train : \n","\tNégatif : ",
              np.abs(np.sum(y_train[y_train == -1])/len(y_train)),"%\n\tPositif : ",
              np.abs(np.sum(y_train[y_train == 1])/len(y_train)),"%")
        print("Répartition dans le dataset de test : \n","\tNégatif : ",
              np.abs(np.sum(y_test[y_test == -1])/len(y_test)),"%\n\tPositif : ",
              np.abs(np.sum(y_test[y_test == 1])/len(y_test)),"%")
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))
        if classifier == 'lr':
            params = {'penalty':['l2'], 'C' : np.linspace(0.6,0.75,30)}
            clf = LogisticRegression(max_iter=2000, class_weight='balanced', multi_class='ovr')
            
        if classifier == 'svc':
            params = {'C' : np.linspace(0.8, 1.1, 5)}
            clf = SVC(class_weight='balanced', max_iter=2000, kernel='linear', decision_function_shape='ovr')
         
        if classifier == 'gb':
            params = {'loss':['deviance','exponential'], 'learning_rate': np.logspace(-2,0,3),
                     'n_estimators':range(150,200,10), 'max_depth':range(50,150,10)}
            clf = GradientBoostingClassifier(max_features='auto')
            
        if classifier == 'mnb' :
            params = {'alpha': np.linspace(0,1,10), 'fit_prior':['True','False'], 'class_prior':[[0.45, 0.3, 0.25], None]}
            clf = MultinomialNB()
            
        GCV = GridSearchCV(clf, params, verbose=9, n_jobs=-1)
        GCV.fit(X_train, y_train)
        print(GCV.best_params_)
        
        
        if classifier == 'lr' :
            model = LogisticRegression(penalty=GCV.best_params_['penalty'],
                                   C=GCV.best_params_['C'], max_iter=2000, multi_class='ovr', class_weight='balanced')
        if classifier == 'svc' :
            model = SVC(C=GCV.best_params_['C'], gamma=1e-15,
                        decision_function_shape='ovr', class_weight='balanced')
    
        if classifier == 'mnb':
            model = MultinomialNB(alpha=GCV.best_params_['alpha'], fit_prior=GCV.best_params_['fit_prior'], 
                                 class_prior=GCV.best_params_['class_prior'])
    
        if classifier == 'gb' :
            model = GradientBoostingClassifier(loss=GCV.best_params_['loss'],
                                               learning_rate=GCV.best_params_['learning_rate'],
                                               n_estimators=GCV.best_params_['n_estimators'],
                                               max_depth=GCV.best_params_['max_depth'],
                                               max_features='auto')
    
    else :
        f = open('sentiment_model.pkl','rb')
        model = pickle.load(f)
        f.close()
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 500, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 500, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        n_samples, vocabulaire = X.shape
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print('Score', np.sum(predictions == y_test) / len(predictions)) 
    f = open('sentiment_model.pkl','wb')
    pickle.dump(model, f)
    f.close()
    print('Model saved')
    return predictions

def cleanTrainDb(collection):
    textCleanPipeline = [{"$group":{"_id":"$text", "dups":{"$push":"$_id"},"count":{"$sum":1}}},{"$match":{"count":{"$gt":1}}}]
    duplicates = []
    count = 0
    try:
        for doc in collection.aggregate(textCleanPipeline) :
            it = iter(doc['dups'])
            next(it)
            for id in it:
                count += 1
                duplicates.append(pym.DeleteOne({'_id':id}))
        if duplicates:
            collection.bulk_write(duplicates)
    except:
        pass
    
    print(count, 'doublons retirés.')
    client.close()

In [78]:
client = pym.MongoClient('localhost', 27018)
#client = pym.MongoClient('localhost', 27017)
collection = client.tweet.train
collection2 = client.tweet.labelised
print(collection.count())
print(collection2.count())

9887
1421


### Test du nettoyage des tweets

In [68]:
a = tweetPreprocessing(collection, retweet=True)

### Test du nettoyage doublons

In [5]:
cleanTrainDb(collection)
cleanTrainDb(collection2)

0 doublons retirés.
0 doublons retirés.


## Train et test du modèle

In [79]:
#a = getSentiments(client, retweet=True, classifier='lr', full_retrain=True)
# attention en cas de retrain périodique on ne prend que les 500 derniers tweets, il faut donc prédire sur moins de 500
#print(len(a[a==1]), len(a[a==0]))
#b = getSentiments(client, retweet=True, classifier='svc', full_retrain=True)
#print(len(b[b==1]), len(b[b==0]))
#c = getSentiments(client, retweet=True, classifier='mnb', full_retrain=True)
#print(len(c[c==1]), len(c[c==0]))
d = getSentiments(client, retweet=True, classifier='gb', full_retrain=True)
print(len(d[d==1]), len(d[d==0]))


Répartition dans le dataset de train : 
 	Négatif :  0.468355678233 %
	Positif :  0.228312302839 %
Répartition dans le dataset de test : 
 	Négatif :  0.443262411348 %
	Positif :  0.256205673759 %
Tweets : 11272 / Mots : 1452
Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV] n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance 
[CV] n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance 
[CV] n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance 
[CV]  n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance, score=0.558580, total=48.0min
[CV] n_estimators=160, learning_rate=0.01, max_depth=50, loss=deviance 
[CV]  n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance, score=0.557067, total=48.4min
[CV] n_estimators=160, learning_rate=0.01, max_depth=50, loss=deviance 


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 48.4min


[CV]  n_estimators=150, learning_rate=0.01, max_depth=50, loss=deviance, score=0.550266, total=48.6min
[CV] n_estimators=160, learning_rate=0.01, max_depth=50, loss=deviance 


KeyboardInterrupt: 

# Essai sur un Xgboost

In [46]:


def TrainXGBoostModel(client, retweet=True, gpuEnabled=False):
    df = tweetPreprocessing(client.tweet.train, 0, retweet)
    df2 = tweetPreprocessing(client.tweet.labelised, 0, retweet)
    try:
        df = pd.concat([df, df2], axis=0, ignore_index=True)
    except:
        pass
    df = df.sample(frac=1.0, replace=False) # mélange des lignes
    X = build_feat_mat(df)
    df[df['sentiment'] == -1] = 2
    y = df['sentiment']
    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.1) # 10 % for validation
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1) # 10 % for test
    n_samples, vocabulaire = X.shape
    print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))

    boostRounds = 10
    # XGBoost data structure
    xgtrain = xgb.DMatrix(x_train.values, y_train.values)
    xgeval = xgb.DMatrix(x_val.values, y_val.values)
    xgtest = xgb.DMatrix(x_test.values)
    params = {'eta': 0.01, 'gamma' : 0.2, 'max_depth': 200, 'num_class':3,
              'lambda': 3, 'alpha': 0.05, 'objective':'multi:softmax', 'eval_metric':'merror', 'eval_metric':'merror'}
     
    
    # Generate the watchlist
    watchlist = [(xgtrain, "train"), (xgeval, "eval")]
    print("Train the XGBoost model...")
    startTime = time.time()
    clf = xgb.train(params, xgtrain, boostRounds, evals = watchlist, 
                    maximize = True, verbose_eval = 100)

    #Make predictions
    print("Make predictions for testing set...")
    predictions_sr = pd.Series(clf.predict(xgtest, ntree_limit = clf.best_iteration))
    return predictions_sr, clf, y_test, x_train.



In [56]:
preds, model,  truth = TrainXGBoostModel(client)

Tweets : 7791 / Mots : 13133
Train the XGBoost model...
[0]	train-merror:0.183864	eval-merror:0.423077
Make predictions for testing set...
